In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from yfapi import YahooFinanceAPI, Interval
import datetime as dt
import wikipedia as wiki

Obtenemos la lista del SP500 de wikipedia.

In [ ]:
html = wiki.page('List of S&P companies').html().encode('UTF-8')
data = pd.read_html(html)[0]
symbol_list = list(data['Symbol'].values)
industry_list = list(data['GICS Sector'].values)
name_list = list(data['Security'].values)

Cargamos los datos de Yahoo Finance a un DataFrame con las 500 empresas.

In [ ]:
dh = YahooFinanceAPI(Interval.DAILY)
start = dt.datetime(2000, 1, 1)
end = dt.datetime(2021, 12, 31)

Esta función carga el dataset por símbolo y agrega los cálculos pertinentes:<br>
`Retorno Gap, Retorno Intra, Variación y Volatilidad.`

In [ ]:
def getSymbolData( p_symbol, p_name, p_industry, p_start, p_end ):
    df = dh.get_ticker_data(p_symbol, start_date=p_start, end_date=p_end)
    df.insert(loc=0, column='Symbol', value=p_symbol)
    df.insert(loc=1, column='Name', value=p_name)
    df.insert(loc=2, column='Industry', value=p_industry)
    
    df['RetGap']   = np.log(df.Close/df.Close.shift(1)).fillna(0)
    
    df['RetIntra'] = np.log(df.Close/df.Open).fillna(0)
    
    df['Var']      = df['Adj Close'].pct_change()
    
    df['Volat']    = df['Var'].rolling(250).std()*100*(250)**0.5
    return df

Esta función devuelve un dataFrame maestro con los datasets de todos los símbolos.

In [ ]:
def getMasterDataframe():
    df_symbols = []
    for i, symbol in enumerate(symbol_list):
        try:
            data = getSymbolData( symbol, name_list[i], industry_list[i], start, end )
            df_symbols.append(data)
        except:
            continue
    
    return pd.concat(df_symbols, ignore_index=True)

Creamos el dataFrame Maestro con todas las empresas.

In [ ]:
df_symbols = getMasterDataframe()

Agregamos columnas que nos serán de utilidad a la hora de graficar.

In [ ]:
#Convertimos la fecha a tipo Date
df_symbols['Date'] = pd.to_datetime(df_symbols['Date'], format='%Y-%m-%d')
#Agregamos el dia de la semana cómo columna
df_symbols['DayOfWk'] = df_symbols['Date'].dt.day_name()
#Agregamos el año cómo columna
df_symbols['Year'] = df_symbols['Date'].dt.year

Exportamos nuestro DataFrame.

In [ ]:
#Exportamos a CSV
df_symbols.to_csv('sp500.csv')